# Mappings around CIS Controls
The [CIS Controls](https://www.cisecurity.org/controls_pre) are a set of best practices for defending against cyber attacks. The controls are practical and are [mapped](https://www.cisecurity.org/controls/cis-controls-navigator/) to and from popular standards like ISO 27001, NIST SP-800 53 and NIST CSF.

In this notebook these mappings are leveraged to provide a basic mapping from MITRE ATT&CK towards these other frameworks, like ISO 27001. At the time of writing, there exists no publicly available mapping from MITRE ATT&CK to ISO 27001, hence this approach. While it is not the optimal approach, and a dedicated mapping would be more accurate, this method is a starting point.

In [1]:
import pandas as pd

EXPORT = False

def get_cis_attck_mapping():
    cis_attck_mapping = pd.read_excel("CIS_Controls_v8_ATTCKv82_condensed.xlsx")
    cis_attck_mapping = cis_attck_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
    cis_attck_mapping['Technique ID'] = cis_attck_mapping['Technique ID'].str.split(', ')
    return cis_attck_mapping.explode('Technique ID')

In [6]:
ttp_control_map = get_cis_attck_mapping()
print("Number of mappings:",len(ttp_control_map.index))
print("Number of TTPs:",len(ttp_control_map['Technique ID'].value_counts().index))

ttp_control_map = ttp_control_map.drop_duplicates()
ttp_control_map

Number of mappings: 2962
Number of TTPs: 383


,Control ID,Control Name,Description,Technique ID
0,1.1,Establish and Maintain Detailed Enterprise Ass...,"Establish and maintain an accurate, detailed, ...",T1200
1,1.2,Address Unauthorized Assets,Ensure that a process exists to address unauth...,T1200
2,1.4,Use Dynamic Host Configuration Protocol (DHCP)...,Use DHCP logging on all DHCP servers or Intern...,T1200
3,2.1,Establish and Maintain a Software Inventory,Establish and maintain a detailed inventory of...,T1560
3,2.1,Establish and Maintain a Software Inventory,Establish and maintain a detailed inventory of...,T1560.001
...,...,...,...,...
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1562.006
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1137.002
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1539
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1535


In [111]:
ttp_control_map.to_excel("")

T1072        29
T1552        28
T1210        28
T1190        24
T1021.001    24
             ..
T1555.001     1
T1498         1
T1203         1
T1218.011     1
T1037.001     1
Name: Technique ID, Length: 383, dtype: int64


Control ID  Control Name                                                                               
4.1         Establish and Maintain a Secure Configuration Process                                          210
18.3        Remediate Penetration Test Findings                                                            169
6.2         Establish an Access Revoking Process                                                           158
6.1         Establish an Access Granting Process                                                           158
18.5        Perform Periodic Internal Penetration Tests                                                    153
                                                                                                          ... 
12.6        Use of Secure Network Management and Communication Protocols                                     1
1.2         Address Unauthorized Assets                                                                      1
4.9     

#### CIS to ISO 27k mapping
Merge the CIS-ATTCK mapping and the CIS-ISO27k mapping and include the control names. Write the result to [an excel file](./ISO_27001_2022_ATTCKv82.xlsx).

In [112]:
cis_iso_mapping = pd.read_excel("CIS_Controls_v8_ISO27k_condensed.xlsx")
cis_iso_mapping = cis_iso_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
cis_iso_mapping['ISO Security Controls'] = cis_iso_mapping['ISO Security Controls'].apply(str).str.split(', ')
print("Number of mappings:",len(cis_iso_mapping.index))
cis_iso_mapping.explode('ISO Security Controls')['ISO Security Controls'].unique().shape

Number of mappings: 133


(62,)

In [113]:
def get_iso_attck():
    cis_iso_mapping = pd.read_excel("CIS_Controls_v8_ISO27k_condensed.xlsx")
    cis_iso_mapping = cis_iso_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
    cis_iso_mapping['ISO Security Controls'] = cis_iso_mapping['ISO Security Controls'].apply(str).str.split(', ')

    cis_attck_mapping = pd.read_excel("CIS_Controls_v8_ATTCKv82_condensed.xlsx")
    cis_attck_mapping = cis_attck_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
    cis_attck_mapping['Technique ID'] = cis_attck_mapping['Technique ID'].str.split(', ')

    cis_attck_iso = cis_attck_mapping.merge(cis_iso_mapping,on="Control ID")
    cis_attck_iso = cis_attck_iso.explode("ISO Security Controls")
    iso_attck = cis_attck_iso[['ISO Security Controls','Technique ID']]
    iso_attck = iso_attck.set_axis(['Control ID','Technique ID'],axis=1)
    iso_attck = iso_attck.explode('Technique ID').drop_duplicates(subset=["Control ID","Technique ID"])
    iso_attck.set_index('Control ID',inplace=True)
    iso_attck.drop_duplicates()
    return iso_attck


iso_attck = get_iso_attck()
print(iso_attck.index.unique().shape)
iso_controls = pd.read_excel("ISO_27001_2022_controls.xlsx", dtype=str)
iso_attck = iso_attck.merge(iso_controls,left_index=True,right_on="Control ID")
iso_attck.set_index('Control ID',inplace=True)

if EXPORT:
    iso_attck.to_excel("ISO_27001_2022_ATTCKv82.xlsx")
iso_attck


(47,)


,Technique ID,Control Name
Control ID,,
5.9,T1200,Inventory of information and other associated ...
5.9,T1560,Inventory of information and other associated ...
5.9,T1560.001,Inventory of information and other associated ...
5.9,T1059.006,Inventory of information and other associated ...
5.9,T1021.005,Inventory of information and other associated ...
...,...,...
8.29,T1574.009,Security testing in development and acceptance
8.29,T1574.007,Security testing in development and acceptance
8.29,T1574.008,Security testing in development and acceptance


# Add ISO 27002:2013 support
Try mapping to ISO 27002:2013 by using an existing mapping between ISO 27002:2022 and ISO 27002:2013

In [114]:
iso_2022_2013 = pd.read_excel("ISO_27001_2022_2013.xlsx",dtype=str)

s = r'A?[.]?(\d+)[.]'
iso_2022_2013['Control-nr. 2013'].value_counts().index.str.extract(s).value_counts().sort_index()

10     2
11    15
12    14
13     7
14    13
15     5
16     7
17     4
18     8
5      2
6      7
7      6
8     10
9     14
dtype: int64

In [115]:
iso_2013_attck = iso_attck.merge(iso_2022_2013,left_index=True,right_on='Control-nr. 2022')
iso_2013_attck.drop(['Control Name','Control-nr. 2022'],axis=1,inplace=True)
iso_2013_attck.columns = ['Technique ID','Control ID', 'Control Name']
iso_2013_attck.set_index('Control ID',inplace=True)
iso_2013_attck = iso_2013_attck.drop_duplicates()
if EXPORT:
    iso_2013_attck.to_excel("ISO_27001_2013_ATTCKv82.xlsx")
iso_2013_attck

,Technique ID,Control Name
Control ID,,
8.1.1,T1200,Inventariseren van bedrijfsmiddelen
8.1.2,T1200,Eigendom van bedrijfsmiddelen
8.1.1,T1560,Inventariseren van bedrijfsmiddelen
8.1.2,T1560,Eigendom van bedrijfsmiddelen
8.1.1,T1560.001,Inventariseren van bedrijfsmiddelen
...,...,...
14.2.9,T1574.008,Systeemacceptatietests
14.2.8,T1574.001,Testen van systeembeveiliging
14.2.9,T1574.001,Systeemacceptatietests


In [116]:
iso_2013_attck.index.map(str).value_counts().index.str.extract(s).value_counts().sort_index()

11     3
12    10
13     6
14     9
15     2
16     4
18     2
5      2
6      4
7      2
8      7
9     14
dtype: int64

# Another approach
Within the CIS v8. to MITRE ATTCK mapping are the CIS v7.1 controls included. There also exist mapping from CIS 7.1 to ISO 27001:2013. This will be leveraged.

In [117]:
cis7_attck = pd.read_excel("CIS_Controls_v7.1_ATTCKv82_condensed.xlsx",dtype=str)
cis7_attck = cis7_attck.dropna().drop_duplicates()
cis7_attck['CIS V7.1'] = cis7_attck['CIS V7.1'].apply(str).str.split(', ')
cis7_attck = cis7_attck.explode('CIS V7.1').drop_duplicates()
print("Number of mappings:",cis7_attck.shape)

Number of mappings: (4619, 2)


In [118]:
cis_iso13_mapping = pd.read_excel("CIS_Controls_v7.1_Mapping_to_ISO_27001_2013_condensed.xlsx",dtype=str)
cis_iso13_mapping = cis_iso13_mapping.drop(["CIS Controls v7.1","CIS Controls","Asset Type"],axis=1)
cis_iso13_mapping['ISO 27001:2013'] = cis_iso13_mapping['ISO 27001:2013'].apply(str).str.split(', ')

cis7_iso13_attck = cis7_attck.merge(cis_iso13_mapping,left_on="CIS V7.1",right_on="CIS Safeguards")
iso13_attck = cis7_iso13_attck.explode("ISO 27001:2013")
iso13_attck = iso13_attck.drop(['CIS V7.1', 'CIS Safeguards'],axis=1)
iso13_attck = iso13_attck.drop_duplicates()
iso13_attck


,TTP ID,ISO 27001:2013
0,T1200,A.8.1.1
3,T1200,A.13.1.3
4,T1200,A.13.1.2
5,T1200,A.11.2.5
7,T1560,A.8.1.1
...,...,...
3880,T1562.006,A.18.2.3
3881,T1137.002,A.18.2.3
3882,T1539,A.18.2.3
3883,T1535,A.18.2.3


In [119]:
import re
s = r'A[.](\d+)[.]'
print(re.search(s,'A.13.').group(1))
mapping_analysis = cis_iso13_mapping.explode('ISO 27001:2013')
mapping_analysis['ISO 27001:2013'].value_counts().index.str.extract(s).value_counts().sort_index()
# mapping_analysis
# print("Number of ISO controls",mapping_analysis['ISO 27001:2013'].unique().shape)
# print("Number of CIS controls",mapping_analysis['CIS Safeguards'].unique().shape)

13


10     1
11     1
12    12
13     4
14     5
16     4
18     1
6      2
7      1
8      6
9     10
dtype: int64

#### Add BIO names

In [120]:
bio_controls = pd.read_excel("ISO_27001_2022_2013.xlsx", usecols=[1,2], dtype=str)
bio_controls = bio_controls.drop_duplicates()
print(bio_controls.columns)
bio_controls['Control-nr. 2013'] = "A."+bio_controls['Control-nr. 2013']
bio_controls

iso13_attck_names = iso13_attck.merge(bio_controls,left_on="ISO 27001:2013",right_on="Control-nr. 2013",how='left')
iso13_attck_names.drop('Control-nr. 2013',axis=1,inplace=True)
iso13_attck_names.columns = ['Technique ID','Control ID','Control Name']
iso13_attck_names.set_index('Control ID',inplace=True)
if EXPORT:
    iso13_attck_names.to_excel("ISO_27001_2013_ATTCKv82_FROM_CIS7.xlsx")
iso13_attck_names


Index(['Control-nr. 2013', 'Control-Name 2013'], dtype='object')


,Technique ID,Control Name
Control ID,,
A.8.1.1,T1200,Inventariseren van bedrijfsmiddelen
A.13.1.3,T1200,Scheiding in netwerken
A.13.1.2,T1200,Beveiliging van netwerkdiensten
A.11.2.5,T1200,Verwijdering van bedrijfsmiddelen
A.8.1.1,T1560,Inventariseren van bedrijfsmiddelen
...,...,...
A.18.2.3,T1562.006,Beoordeling van technische naleving
A.18.2.3,T1137.002,Beoordeling van technische naleving
A.18.2.3,T1539,Beoordeling van technische naleving


In [123]:
iso13_attck_names.index.value_counts().index.str.extract(s).value_counts().sort_index()

10     1
11     1
12    10
13     4
14     5
18     1
6      1
7      1
8      4
9     10
dtype: int64

# Mapping from CIS to NIST CSF
Even though there exists a mapping from NIST SP-800 53 to NIST CSF, it might be interesting to spot the difference.